In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Reading movies file
movies = pd.read_csv('movies_metadata.csv', sep=',', encoding='latin-1')

# Reading ratings file
df_ratings_small = pd.read_csv('ratings_small.csv', sep=',', encoding='latin-1')
#usecols=['userId','movieId','rating','timestamp']
#usecols=['movieId','title','genres']

movies_item=movies

df_ratings_small['movieId'] = df_ratings_small['movieId'].astype('str')

df_ratings_mer = df_ratings_small.merge(movies_item[['title','id']], left_on='movieId', right_on='id')

df_ratings_mer = df_ratings_mer[['userId','movieId','rating', 'title']]

df_ratings_mer.head


/Users/tejasnadkarni/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<bound method NDFrame.head of        userId movieId  rating              title
0           1    1371     2.5          Rocky III
1           4    1371     4.0          Rocky III
2           7    1371     3.0          Rocky III
3          19    1371     4.0          Rocky III
4          21    1371     3.0          Rocky III
...       ...     ...     ...                ...
44989     652  129009     4.0     Love Is a Ball
44990     653    2103     3.0            Solaris
44991     659     167     4.0              K-PAX
44992     659     563     3.0  Starship Troopers
44993     665     129     3.0      Spirited Away

[44994 rows x 4 columns]>

In [2]:
ratings_matrix_items = df_ratings_mer.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix_items.fillna( 0, inplace = True )
ratings_matrix_items.shape

(2830, 671)

In [3]:
ratings_matrix_items

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.metrics import pairwise_distances
movie_similarity = 1 - pairwise_distances(ratings_matrix_items.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix_items = pd.DataFrame( movie_similarity )
ratings_matrix_items

,0,1,2,3,4,5,6,7,8,9,...,2820,2821,2822,2823,2824,2825,2826,2827,2828,2829
0,0.000000,0.0,0.000000,0.0,0.0,0.086712,0.000000,0.099687,0.000000,0.077110,...,0.000000,0.000000,0.0,0.000000,0.143825,0.0,0.102418,0.133132,0.078507,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.578542,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.245026,0.423207,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.196649,0.000000,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.245026,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.282138,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2825,0.000000,0.0,0.000000,0.0,0.0,0.214397,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0
2826,0.102418,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.099829,0.000000,0.030216,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.353784,0.0
2827,0.133132,0.0,0.196649,0.0,0.0,0.222851,0.083223,0.055298,0.245811,0.000000,...,0.245811,0.094808,0.0,0.000000,0.237696,0.0,0.000000,0.000000,0.000000,0.0
2828,0.078507,0.0,0.000000,0.0,0.0,0.039069,0.000000,0.086087,0.000000,0.000000,...,0.000000,0.000000,0.0,0.372046,0.077092,0.0,0.353784,0.000000,0.000000,0.0


In [5]:
def item_similarity(movieName): 
    """
    recomendates similar movies
   :param data: name of the movie 
   """
    try:
        #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
        user_inp=movieName
        inp=movies_item[movies_item['title']==user_inp].index.tolist()
        inp=inp[0]

        movies_item['similarity'] = ratings_matrix_items.iloc[inp]
        movies_item.columns = ['movie_id', 'title', 'release_date','similarity']
    except:
        print("Sorry, the movie is not in the database!")

In [6]:
def recommendedMoviesAsperItemSimilarity(user_id):
    """
     Recommending movie which user hasn't watched as per Item Similarity
    :param user_id: user_id to whom movie needs to be recommended
    :return: movieIds to user 
    """
    user_movie= df_ratings_mer[(df_ratings_mer.userId==user_id) & df_ratings_mer.rating.isin([5,4.5])][['title']]
    user_movie=user_movie.iloc[0,0]
    print(user_movie)
    item_similarity(user_movie)
    
    sorted_movies_as_per_userChoice=movies_item.sort_values(["similarity"], ascending = False )
    print(sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] >=0.45])
    sorted_movies_as_per_userChoice=sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] >=0.45]['movie_id']
    recommended_movies=list()
    df_recommended_item=pd.DataFrame()
    user2Movies= df_ratings[df_ratings['userId']== user_id]['movieId']
    for movieId in sorted_movies_as_per_userChoice:
            if movieId not in user2Movies:
                df_new= df_ratings[(df_ratings.movieId==movieId)]
                df_recommended_item=pd.concat([df_recommended_item,df_new])
            best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
    return best10['movieId']

In [10]:
def movieIdToTitle(listMovieIDs):
    """
     Converting movieId to titles
    :param user_id: List of movies
    :return: movie titles
    """
    movie_titles= list()
    for id in listMovieIDs:
        movie_titles.append(movies_item[movies_item['movie_id']==id]['title'])
    return movie_titles

In [9]:
user_id=105
print("Recommended movies,:\n",movieIdToTitle(recommendedMoviesAsperItemSimilarity(user_id)))

Sleepless in Seattle
Sorry, the movie is not in the database!
     adult belongs_to_collection budget                          genres  \
796  False                   NaN      0  [{'id': 35, 'name': 'Comedy'}]   

    homepage     id    imdb_id original_language        original_title  \
796      NaN  69895  tt0108220                en  Stefano Quantestorie   

                                              overview  ... revenue runtime  \
796  La storia di Stefano (Maurizio Nichetti) sembr...  ...     0.0    82.0   

    spoken_languages    status tagline                 title  video  \
796               []  Released     NaN  Stefano Quantestorie  False   

    vote_average vote_count similarity  
796          6.0        1.0   0.515688  

[1 rows x 25 columns]


KeyError: 'movie_id'